In [1]:
import numpy as np
import pandas as pd
import osmnx as ox
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
from geopy.distance import geodesic
import contextily as ctx
from shapely.geometry import box
from scipy.spatial import cKDTree

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
#Extract the file containing all the data

file_path = 'raw_data-2.csv'

df = pd.read_csv(file_path)

print("The dataframe of all data has the shape:",df.shape)
print("The parameters given by the data are:",df.columns.tolist())

/var/folders/ym/s4xnst_s0y769j7rl84pgpww0000gn/T/ipykernel_1758/3840613656.py:5: DtypeWarning: Columns (8,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


The dataframe of all data has the shape: (340286, 31)
The parameters given by the data are: ['departure', 'arrival', 'route', 'departure_date_num', 'departure_time', 'seats_total', 'seats_booked', 'distance_BBC', 'announce_published_date', 'announce_views', 'arrival_latitude', 'arrival_longitude', 'car_mark', 'car_model', 'comment_average_note', 'commission', 'departure_date', 'departure_latitude', 'departure_longitude', 'departure_strike', 'driver_age', 'driving_quality', 'experience', 'member_since_month', 'member_since_year', 'price_driver', 'price_driver_default', 'price_driver_hcap', 'price_driver_lcap', 'published_ads', 'trip_description']


In [3]:
#Extracting the extrema coordinates for points departing from Paris

paris_df = df[df['departure'] == 'Paris']

In [4]:
def find_nearest_neighbors_osmnx(paris_df, key, value):

    min_lat = paris_df['departure_latitude'].min()
    max_lat = paris_df['departure_latitude'].max()
    min_lon = paris_df['departure_longitude'].min()
    max_lon = paris_df['departure_longitude'].max()

    tags5  = {str(key): str(value)}
    all_stations = ox.geometries_from_bbox(north=max_lat, south=min_lat, east=max_lon, west=min_lon, tags=tags5)

    all_stations_points = all_stations[all_stations.geometry.geom_type == 'Point']

    if not all_stations_points.empty:
        all_stations_points['latitude'] = all_stations_points.geometry.y
        all_stations_points['longitude'] = all_stations_points.geometry.x
        all_stations_points['name'] = all_stations_points.name

    # Convert point_df latitudes and longitudes to a NumPy array (KDTree will be built from this)
    point_coords = np.array(list(zip(all_stations_points['latitude'],  all_stations_points['longitude'])))
    
    # Build the KDTree using point_df coordinates
    point_tree = cKDTree(point_coords)
    
    # Extract latitudes and longitudes from Paris_df for querying
    paris_coords = np.array(list(zip(paris_df['departure_latitude'], paris_df['departure_longitude'])))
    
    # Query the KDTree with the paris_df coordinates to find the nearest neighbors
    _, indices = point_tree.query(paris_coords, k=1)
    
    # Get the coordinates of the nearest points
    nearest_points = point_coords[indices]

    id_points = [all_stations_points['name'][i] for i in indices]
    id_lat = [all_stations_points['latitude'][i] for i in indices]
    id_lon = [all_stations_points['longitude'][i] for i in indices]
    
    # Compute the geodesic distance for each pair of points
    distances = [
        geodesic((paris_lat, paris_lon), (nearest_lat, nearest_lon)).meters
        for (paris_lat, paris_lon), (nearest_lat, nearest_lon) in zip(paris_coords, nearest_points)
    ]
    
    # Return the indices of the nearest neighbors and the distances to those neighbors
    return indices, id_points, distances, id_lat, id_lon


In [5]:
def find_nearest_neighbors_scratch(paris_df, list_lat, list_lon, list_name):

    min_lat = paris_df['departure_latitude'].min()
    max_lat = paris_df['departure_latitude'].max()
    min_lon = paris_df['departure_longitude'].min()
    max_lon = paris_df['departure_longitude'].max()

    all_stations_points = pd.DataFrame()
    all_stations_points['latitude'] = list_lat
    all_stations_points['longitude'] = list_lon
    all_stations_points['name'] = list_name

    # Convert point_df latitudes and longitudes to a NumPy array (KDTree will be built from this)
    point_coords = np.array(list(zip(all_stations_points['latitude'],  all_stations_points['longitude'])))
    
    # Build the KDTree using point_df coordinates
    point_tree = cKDTree(point_coords)
    
    # Extract latitudes and longitudes from Paris_df for querying
    paris_coords = np.array(list(zip(paris_df['departure_latitude'], paris_df['departure_longitude'])))
    
    # Query the KDTree with the paris_df coordinates to find the nearest neighbors
    _, indices = point_tree.query(paris_coords, k=1)
    
    # Get the coordinates of the nearest points
    nearest_points = point_coords[indices]

    id_points = [all_stations_points['name'][i] for i in indices]
    id_lat = [all_stations_points['latitude'][i] for i in indices]
    id_lon = [all_stations_points['longitude'][i] for i in indices]
    
    # Compute the geodesic distance for each pair of points
    distances = [
        geodesic((paris_lat, paris_lon), (nearest_lat, nearest_lon)).meters
        for (paris_lat, paris_lon), (nearest_lat, nearest_lon) in zip(paris_coords, nearest_points)
    ]
    
    # Return the indices of the nearest neighbors and the distances to those neighbors
    return indices, id_points, distances, id_lat, id_lon
